In [29]:
import requests
import keyboard
from bs4 import BeautifulSoup
from datetime import datetime

while True:
    try:
        # 輸入股票代號
        stock_symbol = input("請輸入股票代號：")
        # stock_symbol = 2330

        #跳出迴圈
        if keyboard.is_pressed('esc'):
            print("※按下了Esc鍵, 跳出迴圈※")
            break

        # 股利網頁URL
        url = f'https://tw.stock.yahoo.com/quote/{stock_symbol}/dividend'

        # 發送GET請求
        response = requests.get(url)

        # 解析HTML回應
        soup = BeautifulSoup(response.content, 'html.parser')

        # 抓股票名稱
        name = soup.find('h1', {'class' : 'C($c-link-text) Fw(b) Fz(24px) Mend(8px)'})
        if name == None:
            print('查無此股')
        else:
            name = name.text.strip()

            # 股票現價
            price = soup.find('span', {'class' : 'Fz(32px)'}).text.strip()

            # 找到股利表格
            table = soup.find('div', {'class': 'table-body-wrapper'})

            # 找到股利資料行
            rows = table.find_all('div', {'class': 'table-row'})

            print(f'股票名稱: {name}, 現價: {price}')
            period = 5
            count = period
            total = 0
            current_index = 0
            year_set = set()
            # 顯示前五年的股利資訊
            while True:
                if current_index >= len(rows):
                    break;
                row = rows[current_index]
                current_index = current_index + 1

                columns = row.find_all('div')
                year_str = columns[1].text.strip()
                cash_dividend = columns[2].text.strip()
                stock_dividend = columns[3].text.strip()
                yield_ratio = columns[4].text.strip()
                year_set.add(int(year_str[:4]))
                if len(year_set) <= period:
                    print(f'年份: {year_str}\t現金股利: {cash_dividend}\t股票股利: {stock_dividend}\t殖利率: {yield_ratio}')
                    if cash_dividend != '-':
                        total = total + float(cash_dividend)
                    else:
                        count = count - 1
                else:
                    break;
                
            print('-')
            avgCount = len(year_set) if len(year_set) < count else count #判斷是否資料不足或意外中斷
            avg = round(total/avgCount, 2)
            print(f'平均股利: {avg}')
            print("便宜(x16): {:.2f}".format(avg*16))
            print("平均(x20): {:.2f}".format(avg*20))
            print("太貴(x32): {:.2f}".format(avg*32))
            print('==================================================================')
    except Exception as e:
        print(f"出現錯誤, {e}");
# python -m jupyter nbconvert --to script avgStock.ipynb
# pyinstaller --onefile avgStock.py

股票名稱: 華榮, 現價: 20.25
年份: 2022	現金股利: 0.40	股票股利: 0.00	殖利率: 1.75%
年份: 2021	現金股利: 1.50	股票股利: 0.00	殖利率: 9.71%
年份: 2020	現金股利: 0.70	股票股利: 0.00	殖利率: 2.98%
年份: 2019	現金股利: 0.66	股票股利: 0.00	殖利率: 6.61%
年份: 2018	現金股利: -	股票股利: -	殖利率: -
-
{2017, 2018, 2019, 2020, 2021, 2022} 4 4
平均股利: 0.81
便宜(x16): 12.96
平均(x20): 16.20
太貴(x32): 25.92
※按下了Esc鍵, 跳出迴圈※
